# SAO/LIP Python Primer Course Lecture 8

In this notebook, you will learn about:
- The `scipy` library
- Integration
- Optimization
- Linear algebra
- Statistical distibutions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/acorreia61201/SAOPythonPrimer/blob/main/lectures/Lecture7.ipynb)

## The `scipy` Library

We've covered `numpy`, which is a powerful library in its own right. However, there are some common mathematical problems that would be convenient to have on-hand when doing scientific programming. The `scipy` library is designed specifically for this. It builds upon the functionality of `numpy` to include hundreds of functions that span the breadth of mathematics. Talking about every single thing that you can do with `scipy` would take a summer in itself, so for now we'll cover some of the most notable functionalities of the library.

Of course, the first thing we have to do is make sure it's installed:

In [1]:
!pip install scipy

In [2]:
import scipy

ModuleNotFoundError: No module named 'scipy'

There's so much you can do with `scipy` that it's often more convenient to import individual sublibraries. We'll do this at the start of each relevant section.

## Integration

Differential equations are used throughout STEM to model a variety of problems. The most common way to solve these are with integration, which can be tricky to implement programmatically (as you've probably seen in the exercises). `scipy` has an entire sublibrary `integrate` which provides various integration functions:

In [ ]:
from scipy import integrate